# Study of tags assigned to user prompts

This notebook collects a dataset of user prompts, assigns descriptive tags to them using an LLM-judge, and studies the aggregates of the tags.

# Source of data
```sql
select 
  u.user_id, u.email, u.country_code,
  cm.message_id, cm.message_type, cm.content, cm.created_at, cm.language_code, cm.annotations, cm.completion_status,
  t.turn_id, t.sequence_id as turn_sequence_id,
  tq.prompt_difficulty, tq.prompt_is_safe,
  lm.internal_name as lm_internal_name,
  c.name as category_name
from chat_messages cm
join turns t on t.turn_id = cm.turn_id 
join users u on t.creator_user_id = u.user_id 
left join turn_qualities tq on tq.turn_id = t.turn_id 
left join categories c on c.category_id = cm.category_id 
left join language_models lm on lm.language_model_id = cm.assistant_language_model_id 
```

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import logging
import json
from collections import Counter
from scipy.stats import chi2_contingency, power_divergence
import numpy as np

from langchain_core.language_models import BaseChatModel
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate

import ypl.db.all_models
from ypl.backend.llm.labeler import InputType, LLMLabeler, OnErrorBehavior, OutputType
from ypl.backend.llm.judge import TruncatedPromptLabeler
from ypl.backend.config import settings
from ypl.backend.llm.constants import ChatProvider
from ypl.backend.llm.model_data_type import ModelInfo
from ypl.backend.llm.vendor_langchain_adapter import GeminiLangChainAdapter
logging.getLogger().setLevel(logging.WARNING)

In [2]:
# Can use a simple, cheap model for this.
GEMINI_15_FLASH_LLM = GeminiLangChainAdapter(
    model_info=ModelInfo(
        provider=ChatProvider.GOOGLE,
        model="gemini-1.5-flash",
        api_key=settings.GOOGLE_API_KEY,
    ),
    model_config_=dict(
        project_id=settings.GCP_PROJECT_ID,
        region=settings.GCP_REGION,
        temperature=0.0,
        max_output_tokens=256,
        top_k=1,
    ),
)

JUDGE_PROMPT = """
You are a model that assigns "tags" summarizing the user's prompt, as a json with two arrays of strings:
- "high_level_tags": an array of high-levels tags that describe the prompt, such as "weather", "news", "code", etc.
- "detailed_tags": an array of detailed tags that describe the prompt.
tags are ideally 1-3 words, and should be short, descriptive, and relevant. Do not include any markup, just the json array.
Example output:
{{"high_level_tags": ["weather"], "detailed_tags": ["climate patterns", "temperature", "rainfall"]}}

Assign high_level_tags and detailed_tags to the following prompt:

{prompt}
"""


class PromptLabeler(TruncatedPromptLabeler[dict[str, list[str]]]):
    cached = True

    def _prepare_llm(self, llm: BaseChatModel) -> BaseChatModel:
        cp_template = ChatPromptTemplate.from_messages([("human", JUDGE_PROMPT)])
        return cp_template | llm  # type: ignore

    def _parse_output(self, output: BaseMessage) -> dict[str, list[str]]:
        content = output.content.replace("```json", "").replace("```", "").strip()
        return json.loads(content)

    @property
    def error_value(self) -> list[str]:
        return []

labeler = PromptLabeler(GEMINI_15_FLASH_LLM, max_prompt_len=500)

In [3]:
# Load the raw data, which comes from the SQL query above.
df_all = pd.read_csv('messages.csv')

# Exclude yuppsters.
df = df_all[~df_all.email.str.contains("yupp.ai")]

In [4]:
def print_stats(df: pd.DataFrame):
    def print_counts(df: pd.DataFrame, col: str):
        print(f"{col}:")
        for v, c in df[col].value_counts().head().items():
            print(f"  {v}: {c}")
        print()
    print(f"Messages: {len(df)}")
    print_counts(df, "message_type")
    print_counts(df, "category_name")
    print_counts(df, "language_code")
    print_counts(df, "prompt_difficulty")
    print_counts(df, "email")
    
print("--- Entire dataset ---")
print_stats(df_all)

print("--- Excluding yuppsters ---")
print_stats(df)

--- Entire dataset ---
Messages: 95769
message_type:
  ASSISTANT_MESSAGE: 49987
  USER_MESSAGE: 22916
  QUICK_RESPONSE_MESSAGE: 22866

category_name:
  Factual: 5852
  Other: 3776
  Code: 2891
  Creative Writing: 1764
  Education: 1480

language_code:
  EN: 91702
  FR: 671
  ID: 491
  ES: 312
  IT: 246

prompt_difficulty:
  7.0: 20675
  8.0: 15430
  3.0: 11060
  4.0: 9371
  1.0: 8561

email:
  gilad@yupp.ai: 6664
  mnn@yogins.com: 4385
  bhanu@yupp.ai: 3941
  w@yupp.ai: 3455
  minqi@yupp.ai: 3312

--- Excluding yuppsters ---
Messages: 56991
message_type:
  ASSISTANT_MESSAGE: 30041
  USER_MESSAGE: 13485
  QUICK_RESPONSE_MESSAGE: 13465

category_name:
  Factual: 3126
  Other: 1802
  Code: 1778
  Creative Writing: 1036
  Education: 1033

language_code:
  EN: 54323
  FR: 546
  ID: 478
  ES: 221
  LT: 183

prompt_difficulty:
  7.0: 15867
  8.0: 13307
  4.0: 6428
  3.0: 5981
  9.0: 3776

email:
  mnn@yogins.com: 4385
  babak.hamadani@gmail.com: 1862
  tysreddy@gmail.com: 1711
  adithya.vadap

In [5]:
# Flag to skip the LLM-based tagging step, if it was already run and stored locally.
PRE_CALC_PROMPTS = True

if not PRE_CALC_PROMPTS:
    prompts = df[
        (df.message_type == "USER_MESSAGE") & (df.content.str.len() > 5)
    ]

    res = await labeler.abatch_label(prompts.content.tolist(), num_parallel=4)

    def get_tags(result, col):
        return result.get(col, []) if isinstance(result, dict) else []

    prompts["high_level_tags"] = [get_tags(x, "high_level_tags") for x in res]
    prompts["detailed_tags"] = [get_tags(x, "detailed_tags") for x in res]

    with open("prompts.csv", "w") as f:
        prompts.to_csv(f, index=False)

prompts = pd.read_csv("prompts.csv")
prompts["high_level_tags"] = prompts["high_level_tags"].apply(eval)
prompts["detailed_tags"] = prompts["detailed_tags"].apply(eval)

In [6]:
def flatten_tags(df: pd.DataFrame, col: str):
    return [tag for tags in df[col] for tag in tags]

all_tags = {
    col: flatten_tags(prompts, col) for col in ["high_level_tags", "detailed_tags"]
}
all_counts = {
    col: Counter(all_tags[col]) for col in all_tags
}

def analyze_tags(df: pd.DataFrame, tag_col: str):
    # Calculate tag counts for df.
    tags = flatten_tags(df, tag_col)
    counts = Counter(tags)
    
    # Display the most common tags.
    common_tags = dict(counts.most_common(30))
    
    # Totals in the subset and overall dataset.
    total = sum(counts.values())
    total_all = sum(all_counts[tag_col].values())
    total_rest = total_all - total
    
    # Dictionaries to store divergence scores.
    chi2_scores = {}
    llr_chi2_scores = {}
    
    # Consider all tags present either in the overall counts or in the df.
    tokens = set(all_counts[tag_col].keys()).union(counts.keys())
    for token in tokens:
        count = counts.get(token, 0)
        count_overall = all_counts[tag_col].get(token, 0)
        count_rest = count_overall - count
        
        # Build 2x2 contingency table.
        table = [
            [count, count_rest],
            [total - count, total_rest - count_rest]
        ]
        
        # Pearson's chi-square test.
        try:
            chi2, p_value, dof, expected = chi2_contingency(table, correction=False)
            chi2_scores[token] = (chi2, p_value, dof, expected)
        except Exception as e:
            pass
        
    # Sort tags by divergence scores (higher score = more differing).
    sorted_chi2 = sorted(chi2_scores.items(), key=lambda x: x[1][0], reverse=True)
    
    # Print the top diverging tags.
    for token, (score, p_value, dof, expected) in sorted_chi2[:30]:
        obs = counts.get(token, 0)
        overall = all_counts[tag_col].get(token,0)
        obs_pct = obs / total
        overall_pct = overall / total_all
        # Format with fixed width using ljust() for consistent alignment
        token_str = f"{token}".ljust(25)
        stats_str = f"chi2={score:.2f}".ljust(15)
        counts_str = f"obs={obs} ({obs_pct:.3%}) overall={overall} ({overall_pct:.3%})"
        # print(f"  {token_str}{stats_str}{counts_str}")
    return common_tags, {token: v[0] for token, v in sorted_chi2[:30]}

In [7]:
data = {}
for category in prompts.category_name.unique():
    category_df = prompts[prompts.category_name == category]
    top_detailed_tags, chi2_detailed_tags = analyze_tags(category_df, "detailed_tags")
    top_high_level_tags, chi2_high_level_tags = analyze_tags(category_df, "high_level_tags")
    data[category] = {
        "top_detailed_tags": top_detailed_tags,
        "chi2_detailed_tags": chi2_detailed_tags,
        "top_high_level_tags": top_high_level_tags,
        "chi2_high_level_tags": chi2_high_level_tags,
    }
    print(category, end="...")
print()


Opinion...Other...Factual...Education...Math...Analysis...Creative Writing...Summarization...Comparison...Multilingual...Code...Advice...Reasoning...Entertainment...nan...


In [8]:
# Create CSVs from the chi2_detailed_tags data, for loading to excel
detailed_tags = {
    category: list(data[category]['chi2_detailed_tags'].keys())
    for category in data.keys()
    if not pd.isna(category)
}
detailed_tags_df = pd.DataFrame(detailed_tags)
detailed_tags_df.to_csv("detailed_tags.csv", index=False)

high_level_tags = {
    category: list(data[category]['chi2_high_level_tags'].keys())
    for category in data.keys()
    if not pd.isna(category)
}
high_level_tags_df = pd.DataFrame(high_level_tags)
high_level_tags_df.to_csv("high_level_tags.csv", index=False)